# Data Analysis

In this Jupyter Notebook we will use SQLite to analyse data from IMDb, saved in the IMDb.db database. This database is composed by 4 tables - <b>basics</b> (title of the movie/show, type of program, year, runtime, ...), <b>ratings</b> (average rating, number of votes), <b>crew</b> (directors, writers), and <b>names</b> (name of the person, birth year, ...). 

In [1]:
%load_ext sql
%sql sqlite:///IMDb.db

We can start to analyse this database by looking at the first 5 values of each table:

### Basics Table

In [2]:
%%sql
SELECT * from basics
limit 5

 * sqlite:///IMDb.db
Done.


index,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


As we can see, the main table contains information about:

- unique identifier of the program (tconst);
- type of program;
- primary title;
- original title;
- whether it is an adult program or not (0 for non-adult and 1 for adult);
- the release year (startYear);
- the end year of TV series (\N for other types);
- the runtime, in minutes;
- the genre, showing up to 3 different types.

### Ratings Table

In [3]:
%%sql
SELECT * from ratings
limit 5

 * sqlite:///IMDb.db
Done.


index,tconst,averageRating,numVotes
0,tt0000001,5.7,1910
1,tt0000002,5.8,256
2,tt0000003,6.5,1712
3,tt0000004,5.6,169
4,tt0000005,6.2,2527


The ratings table can be joined to the Basics table through the <i>tconst</i> variable. Besides that, it contains two other parameters:

- average rating, from 1 to 10;
- the number of votes in each program.

### Crew Table

In [4]:
%%sql
SELECT * from crew
limit 5

 * sqlite:///IMDb.db
Done.


index,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N


Next, we have the crew table, which can also be joined with the previous tables through the tconst variable. It contains information about:

- directors of a given program;
- writers of a given program.

### Names Table

In [5]:
%%sql
SELECT * from names
limit 5

 * sqlite:///IMDb.db
Done.


index,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0050419,tt0072308,tt0053137"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0117057,tt0071877,tt0038355,tt0037382"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0054452,tt0049189,tt0056404,tt0057345"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0077975,tt0080455,tt0072562,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0060827,tt0083922,tt0050986,tt0050976"


Finally, we have the names table, which can be joined with the previou table (crew) through the nconst variable. It contains information about directors, writers, and actors:

- name;
- year of birth;
- death year (if applicable);
- profession;
- titles the person is known for.

## Counting records

Now that we know the structure of each table, it would be interesting to know how many distinct records each table contains:

In [6]:
%%sql
SELECT COUNT(DISTINCT basics.tconst) AS BasicsCount, COUNT(DISTINCT ratings.tconst) AS RatingsCount, COUNT(DISTINCT crew.tconst) AS CrewCount FROM basics
LEFT JOIN ratings ON basics.tconst = ratings.tconst
LEFT JOIN crew ON basics.tconst = crew.tconst

 * sqlite:///IMDb.db
Done.


BasicsCount,RatingsCount,CrewCount
9214174,1259347,9214174


From this we conclude that there are 921474 unique titles in this database and all of them have information about the directors and/or writers. However, only 1259347 titles have a rating associated.

Next, we can group the basics table according to the <i>titleType</i> and see how many records each type contains:

In [7]:
%%sql
SELECT titleType, COUNT(*) AS count, COUNT(*) * 100 / SUM(COUNT(*)) OVER() as percentage FROM basics
GROUP BY titleType
ORDER BY count DESC

 * sqlite:///IMDb.db
Done.


titleType,count,percentage
tvEpisode,6945426,75
short,888469,9
movie,620820,6
video,265066,2
tvSeries,230803,2
tvMovie,137524,1
tvMiniSeries,45047,0
tvSpecial,38220,0
videoGame,32134,0
tvShort,10663,0


We can see that the IMDb database is mainly composed by TV episodes (75% of records), followed by short movies (9% of records) and movies (6% of records).

It would also be interesting to see how many of these records have been rated. We can do a left join of the basics table with the ratings table, connecting both tables with the tconst variable.

In [8]:
%%sql
SELECT basics.titleType, COUNT(ratings.tconst) as count, COUNT(ratings.tconst) * 100 / SUM(COUNT(ratings.tconst)) OVER() as percentage from basics
LEFT JOIN ratings 
ON basics.tconst = ratings.tconst
GROUP BY basics.titleType
ORDER BY count DESC

 * sqlite:///IMDb.db
Done.


titleType,count,percentage
tvEpisode,610982,48
movie,282044,22
short,143337,11
tvSeries,83314,6
video,50317,3
tvMovie,48888,3
tvMiniSeries,13851,1
videoGame,13610,1
tvSpecial,10537,0
tvShort,2467,0


If we focus on the rated records, we see that they are again mainly composed of TV episodes (but this time with a lower percentage of 48%), followed by movies (22%), short movies (11%), and TV series (6%). 

Based on this results, we can focus our attention on TV episodes, movies, short movies, and TV series, which hold a higher number of records and ratings. 

Next, we can see how many rated programs have been released each year, by grouping the data based on the release year and counting the number of records in each year.

In [9]:
%%sql
SELECT basics.startYear, COUNT(*) AS count, COUNT(*) * 100 / SUM(COUNT(*)) OVER() as percentage FROM basics
INNER JOIN ratings
ON basics.tconst = ratings.tconst
WHERE (basics.titleType = 'tvEpisode'OR basics.titleType = 'movie' OR basics.titleType = 'short' OR basics.titleType = 'tvSeries') 
AND basics.startYear != '\N'
GROUP BY startYear
ORDER BY startYear DESC

 * sqlite:///IMDb.db
Done.


startYear,count,percentage
2023,2,0
2022,25225,2
2021,46519,4
2020,47464,4
2019,53603,4
2018,52225,4
2017,51537,4
2016,49949,4
2015,45619,4
2014,43867,3


It is interesting to note that we have programs released as yearly as 1874. Nevertheless, the majority of programs were released between 1997 and 2021.

## Ratings

Since my favourite movie genre is drama, I am curious to see the top 10 drama movies that had the highest ratings since 1997, with at least 100000 votes.

We can get that information by joining the basics and ratings tables through the tconst variable and then add our conditions (startYear >= 1997, titleType = 'movie', etc.)

In [10]:
%%sql
SELECT basics.originalTitle, basics.startYear, basics.genres, ratings.averageRating FROM basics
INNER JOIN ratings
ON basics.tconst = ratings.tconst
WHERE basics.startYear >= 1997
AND basics.genres LIKE '%drama%'
AND basics.titleType = 'movie'
AND ratings.numVotes >= 100000
ORDER BY ratings.averageRating DESC, basics.startYear DESC
LIMIT 10

 * sqlite:///IMDb.db
Done.


originalTitle,startYear,genres,averageRating
The Dark Knight,2008,"Action,Crime,Drama",9.0
The Lord of the Rings: The Return of the King,2003,"Action,Adventure,Drama",9.0
Jai Bhim,2021,"Crime,Drama,Mystery",8.9
The Lord of the Rings: The Two Towers,2002,"Action,Adventure,Drama",8.8
The Lord of the Rings: The Fellowship of the Ring,2001,"Action,Adventure,Drama",8.8
Fight Club,1999,Drama,8.8
Soorarai Pottru,2020,Drama,8.7
Interstellar,2014,"Adventure,Drama,Sci-Fi",8.6
Cidade de Deus,2002,"Crime,Drama",8.6
The Green Mile,1999,"Crime,Drama,Fantasy",8.6


It is interesting to see that all "The Lord of the Rings" movies are in the top 10!

We can also take a look at the highest ranked comedy and drama TV series, since 1997. 

In [11]:
%%sql
SELECT basics.originalTitle, basics.startYear, basics.genres, ratings.averageRating FROM basics
INNER JOIN ratings
ON basics.tconst = ratings.tconst
WHERE basics.startYear >= 1997
AND (basics.genres LIKE '%comedy%' OR basics.genres LIKE '%drama%')
AND basics.titleType = 'tvSeries'
AND ratings.numVotes >= 100000
ORDER BY ratings.averageRating DESC, basics.startYear DESC
LIMIT 10

 * sqlite:///IMDb.db
Done.


originalTitle,startYear,genres,averageRating
Breaking Bad,2008,"Crime,Drama,Thriller",9.5
The Heroes,2008,"Animation,Drama,History",9.4
Scam 1992: The Harshad Mehta Story,2020,"Biography,Crime,Drama",9.3
The Wire,2002,"Crime,Drama,Thriller",9.3
Aspirants,2021,Drama,9.2
Rick and Morty,2013,"Adventure,Animation,Comedy",9.2
Game of Thrones,2011,"Action,Adventure,Drama",9.2
The Sopranos,1999,"Crime,Drama",9.2
Sherlock,2010,"Crime,Drama,Mystery",9.1
Death Note: Desu nôto,2006,"Animation,Crime,Drama",9.0


Most of the TV series of this list are categorized as drama, even thought we were looking for drama and comedy. Only Rick and Morty is categorized as comedy and present in this list. From this, we can deduce that drama series have overall higher rankings that comedy series. Besides that, we also observe that crime is another common genre in this list.

## Directors

To complete this analysis, we will take a look into the crew and names tables.
We can start by looking at the directors of the top 5 drama movies shown before. 

In [12]:
%%sql
SELECT basics.originalTitle, basics.startYear, basics.genres, ratings.averageRating, names.primaryName FROM basics
INNER JOIN ratings
ON basics.tconst = ratings.tconst
INNER JOIN crew
ON basics.tconst = crew.tconst
INNER JOIN names
ON names.nconst = crew.directors
WHERE basics.startYear >= 1997
AND basics.genres LIKE '%drama%'
AND basics.titleType = 'movie'
AND ratings.numVotes >= 100000
ORDER BY ratings.averageRating DESC, basics.startYear DESC
LIMIT 5

 * sqlite:///IMDb.db
Done.


originalTitle,startYear,genres,averageRating,primaryName
The Dark Knight,2008,"Action,Crime,Drama",9.0,Christopher Nolan
The Lord of the Rings: The Return of the King,2003,"Action,Adventure,Drama",9.0,Peter Jackson
Jai Bhim,2021,"Crime,Drama,Mystery",8.9,T.J. Gnanavel
The Lord of the Rings: The Two Towers,2002,"Action,Adventure,Drama",8.8,Peter Jackson
The Lord of the Rings: The Fellowship of the Ring,2001,"Action,Adventure,Drama",8.8,Peter Jackson


Since Christopher Nolan directed the drama movie which obtained the highest average rating, we can conclude this notebook with an analysis of all his movies and respetive rankings.

In [13]:
%%sql
SELECT basics.originalTitle, basics.startYear, basics.genres, ratings.averageRating FROM basics
INNER JOIN ratings
ON basics.tconst = ratings.tconst
INNER JOIN crew
ON basics.tconst = crew.tconst
INNER JOIN names
ON names.nconst = crew.directors
WHERE names.primaryName = 'Christopher Nolan'
ORDER BY basics.startYear

 * sqlite:///IMDb.db
Done.


originalTitle,startYear,genres,averageRating
Larceny,1996,Short,6.1
Doodlebug,1997,"Action,Fantasy,Mystery",7.0
Following,1998,"Crime,Mystery,Thriller",7.5
Memento,2000,"Mystery,Thriller",8.4
Insomnia,2002,"Drama,Mystery,Thriller",7.2
Batman Begins,2005,"Action,Crime,Drama",8.2
The Prestige,2006,"Drama,Mystery,Sci-Fi",8.5
The Dark Knight,2008,"Action,Crime,Drama",9.0
Inception,2010,"Action,Adventure,Sci-Fi",8.8
The Dark Knight Rises,2012,"Action,Drama",8.4
